In [2]:
import datetime
from datetime import timedelta

# get today's date
dateToday = datetime.datetime.today()

daysOfPull = 30

# minus 20 days from the date
startDate = dateToday - timedelta(days=daysOfPull)

print(startDate)

2022-12-06 22:23:54.836233


In [6]:

# Get data from NEWS API

# %pip install newsapi-python

from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='913098f1a0fd429a88cee464f86f2201')


# /v2/everything
data = []
for i in range(30):
    dateToUse = startDate + timedelta(days=i)
    date = dateToUse.strftime("%Y-%m-%d")
    all_articles = newsapi.get_everything(sources='breitbart-news,fox-news,msnbc,reuters,the-huffington-post,the-verve,bbc',from_param=date,to=date,language='en',sort_by='relevancy',page=5)
    data.append(all_articles['articles'])


In [7]:
print(data[0][0]['source']['name'])

Fox News


In [8]:
convertedData = []
for i in range(len(data)):
    for j in range(len(data[i])):
        convertedData.append({"sentence": data[i][j]['description'], "pub_year": data[i][j]['publishedAt'][0:4], "news_link": data[i][j]['url'], "outlet" : data[i][j]['source']['name']})
        # convertedData.append(data[i]['articles'][0])

print(len(convertedData))

# use pandas to create dataframe
import pandas as pd

1796


In [9]:
import numpy as np

print(convertedData[200])

{'sentence': 'President Joe Biden on Thursday celebrated the news that his administration had successfully secured the release of Brittney Griner.', 'pub_year': '2022', 'news_link': 'https://www.breitbart.com/politics/2022/12/08/joe-biden-brittney-griner-incomparable-athlete-represents-best-america/', 'outlet': 'Breitbart News'}


In [10]:
# add converted data to new dataframe
df = pd.DataFrame()
df = pd.DataFrame(convertedData)
df.head()

,sentence,pub_year,news_link,outlet
0,Mevlut Cavusoglu has declared Finland must lif...,2022,https://www.foxnews.com/world/finland-must-lif...,Fox News
1,Arizona Rep. Andy Biggs is challenging Minorit...,2022,https://www.foxnews.com/politics/andy-biggs-to...,Fox News
2,Far-left billionaire George Soros is tied to a...,2022,https://www.foxnews.com/media/george-soros-inf...,Fox News
3,'The Five' co-hosts discuss how the left's cli...,2022,https://www.foxnews.com/media/left-ripped-push...,Fox News
4,Police in Illinois believe evidence suggests a...,2022,https://www.foxnews.com/us/illinois-police-evi...,Fox News


In [11]:
# Save to csv
path = 'api_data_'+ str(len(convertedData)) + '.csv'
df.to_csv(path, index=False)

In [12]:
# import saved prediction model from local file
# %pip install tensorflow==2.10.0 --user

import tensorflow as tf
# show version of tensorflow
print(tf.__version__)
# show version of keras
print(tf.keras.__version__)


2.10.0
2.10.0


In [14]:
model = tf.keras.models.load_model('C://users/verni/Desktop/AI course/group-07/notebooks/bias_model/prev_versions/model')

In [23]:
#import model
from transformers import DistilBertTokenizerFast
import pandas as pd
import numpy as np

# Load model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', do_lower_case=True)
def tokenize(sentences, max_length=218, padding='max_length'):

    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

# Load data
df = pd.read_csv(path)

#add a column for predictions
df['pred'] = ""
df.head()

# Remove a tags from sentences
df['sentence'] = df['sentence'].str.replace('<a.*?>', '')

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
# for each item in the dataframe
for i in range(len(df)):
    # tokenize the sentence
    inputs = dict(tokenizer([df.sentence[i]], return_tensors='tf'))
    prediction = model.predict(inputs)
    df.pred[i] = tf.nn.softmax(prediction['logits'], axis=1).numpy()


1/1 [==============================] - 0s 84ms/step


C:\Users\verni\AppData\Local\Temp\ipykernel_11964\2253884047.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentence'] = df['sentence'].str.replace('<a.*?>', '')
C:\Users\verni\AppData\Local\Temp\ipykernel_11964\2253884047.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.pred[i] = tf.nn.softmax(prediction['logits'], axis=1).numpy()


1/1 [==============================] - 0s 83ms/step


In [17]:
print(df.head())

                                            sentence  pub_year  \
0  Surveillance video shows Austin, Texas, attorn...      2022   
1  Rapper and entrepreneur Kanye West praised Hit...      2022   
2  MSNBC political commentator and former White H...      2022   
3  Elon Musk's brain chip company Neuralink aims ...      2022   
4  Rep. Michael Waltz (R) discusses the lockdown ...      2022   

                                           news_link          outlet  \
0  https://www.breitbart.com/politics/2022/12/01/...  Breitbart News   
1  https://www.breitbart.com/politics/2022/12/01/...  Breitbart News   
2  https://www.breitbart.com/clips/2022/12/01/psa...  Breitbart News   
3  https://www.breitbart.com/tech/2022/12/01/elon...  Breitbart News   
4  https://www.breitbart.com/podcast/2022/12/01/b...  Breitbart News   

                        pred  
0  [[0.8682157, 0.13178426]]  
1   [[0.3139803, 0.6860197]]  
2   [[0.4305216, 0.5694784]]  
3  [[0.9005478, 0.09945217]]  
4  [[0.8563726,

In [25]:
# consider pred and add label to dataframe
df['label'] = ""
for i in range(len(df)):
    if df.pred[i][0][0] > df.pred[i][0][1]:
        if df.pred[i][0][0] > 0.8:
            df.label[i] = 0
    else:
        if df.pred[i][0][1] > 0.8:
            df.label[i] = 1

# print amount of articles with label 0 and 1
print(df.label.value_counts())


C:\Users\verni\AppData\Local\Temp\ipykernel_11964\4023064833.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.label[i] = 0
C:\Users\verni\AppData\Local\Temp\ipykernel_11964\4023064833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.label[i] = 1


0    1449
1     223
      124
Name: label, dtype: int64


In [21]:
# remove entries where label is empty
df = df[df.label != '']

print(df.head())

                                            sentence  pub_year  \
0  Mevlut Cavusoglu has declared Finland must lif...      2022   
1  Arizona Rep. Andy Biggs is challenging Minorit...      2022   
6  Playboy founder Hugh Hefner and former Lakers ...      2022   
7  Are your family members snooping around your s...      2022   
8  Vanna White revealed she "doesn't want to thin...      2022   

                                           news_link    outlet  \
0  https://www.foxnews.com/world/finland-must-lif...  Fox News   
1  https://www.foxnews.com/politics/andy-biggs-to...  Fox News   
6  https://www.foxnews.com/sports/playboy-founder...  Fox News   
7  https://www.foxnews.com/tech/how-keep-your-ama...  Fox News   
8  https://www.foxnews.com/entertainment/wheel-of...  Fox News   

                          pred label  
0  [[0.98699945, 0.013000596]]     0  
1   [[0.9865341, 0.013465842]]     0  
6   [[0.95292354, 0.04707642]]     0  
7   [[0.012375448, 0.9876245]]     1  
8  [[0.9729

In [22]:
# if label is 0 or 1, add to csv file
print(len(df))
df.to_csv('pseudo_labeled_data.csv', index=False)


1666
